In [1]:
from ast import Load
from email import header
from fileinput import filename
from genericpath import exists
from operator import pos, truediv
from pathlib import Path
import sys
import os
from tkinter import W
from turtle import color, title
import numpy as np
import pickle
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re
import csv
import sklearn.model_selection
from collections import Counter
from matplotlib import pyplot as plt
import gensim
from sklearn import svm
from sklearn.metrics import accuracy_score
from matplotlib.pyplot import clf
from sklearn import datasets
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_validate

In [23]:
#Some helpful functions which are repeated throughout the code

In [2]:
#Loads a Pickle file only
def LoadFile(Filename):
    Data = {} #empty dictionary to hold the dataframe
    if exists(Filename): #check if the filepath exists
        Data = pd.read_pickle(Filename)
        return(Data)
    else:
        print("Directory not found!") #error message

In [3]:
#Saves a Pickle file only
def SaveFile(DataFrame,Filename): #save a dataframe to pickle format
    if not exists(Filename): #if the file doesn't already exist
        DataFrame.to_pickle(Filename)

In [4]:
#Opens a tsv file only
def OpenFile(Filename): 
    if exists(Filename): #if the file path exists
        Data = pd.read_csv(Filename,sep='\t',lineterminator='\n') #we call read_csv but with separator parameter tab so in reality we read a tsv
        return Data
    else:
        print("Requested File does not exist!") #error message

In [5]:
def autocpt(pct,allvalues):
    absolute = int(pct / 100.*sum(allvalues))
    return "{:.1f}%\n({:d} g)".format(pct, absolute)

In [6]:
#Splits a tuple
def SplitTuple(TupleArray):
    TupleStr = []
    TupleValues = []
    for items in TupleArray:
        for moreitems in items:
            if isinstance(moreitems,str): #if the items are of type string
                TupleStr.append(moreitems) #append them to the Str list
            else: #else they are of type float
                TupleValues.append(moreitems) #append them to the values list
    return TupleStr,TupleValues #return the set

In [7]:
#Creates a Pie for the Data Analysis part
def CreatePlot(ExplodeData,Colors,Values,StrArray,LegendPreview,LegendTitle):
    Fig,Ax = plt.subplots(figsize=(12,9)) #set the size of the pie
    WedgeProperties = { 'linewidth' : 1, 'edgecolor' : "green" }
    wedges,texts,autotexts = Ax.pie(Values,autopct=lambda pct: autocpt(pct, Values),
                                    explode=ExplodeData,labels=StrArray,shadow=True,
                                    colors=Colors,startangle=90,wedgeprops=WedgeProperties,
                                    textprops=dict(color = "black")) #values that change the pie
    Ax.legend(wedges,LegendPreview,title = "Legend",loc= 'right', 
              bbox_to_anchor = (1,0,0.5,1)) #Parameters for the legend
    plt.setp(autotexts,size = 7,weight = "bold") #parameters for the texts inside the pie
    plt.title(LegendTitle,y=1.08) #Legend Location
    plt.show() #show the pie

In [30]:
#Clean the text in the tweets

In [14]:
    DataFrame = LoadFile("eclass_all_with_sentiment_v2.pkl")#Load the Dataframe
    DataFrame = DataFrame.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
    DataFrame['text'] = DataFrame['text'].str.lower() #make all lowercase
    stop  = stopwords.words('english') 
    pat = r'\b(?:{})\b'.format('|'.join(stop))
    DataFrame['text'] = DataFrame['text'].str.replace(pat, '',regex = True)
    DataFrame['text'] = DataFrame['text'].str.replace(r'\s+', ' ',regex = True)
    cleaned = []
    elements = list(DataFrame['text'])
    for i in elements:
        FlteredText = re.sub('https?://[A-Za-z0-9./]+','',i)
        FlteredText = re.sub("[^a-zA-Z0-9]", " ",FlteredText)
        cleaned.append(re.sub(r'^RT[\s]+', '', FlteredText))
    DataFrame['text'] = cleaned

In [29]:
#Data Analysis

In [ ]:
    #Data analysis 1
    Neg = DataFrame.groupby('sentiment').get_group("NEG") #Create dataframes for each of the sentiment group
    Pos = DataFrame.groupby('sentiment').get_group("POS")
    Neu = DataFrame.groupby('sentiment').get_group("NEU")
    DataStr = []
    DataValues = []
    DataStr.append("NEG") #append the strs NEG POS NEU so they appear in the legend
    DataStr.append("POS")
    DataStr.append("NEU")
    DataValues.append(len(Neg)) #we append the length of each dataframe so we can find the percentage that each sentiment hold in the dataframe
    DataValues.append(len(Pos))
    DataValues.append(len(Neu))
    ExplodeData = (0.1, 0.0, 0.2)
    Colors = ( "brown", "cyan", "orange") #some nice colors
    Preview = "Distribution of sentiments in the tweets" #a bad title
    CreatePlot(ExplodeData,Colors,DataValues,DataStr,DataStr,Preview) #call the create plot to handle the rest

In [ ]:
    #Data analysis 2
    MostCommon = Counter(" ".join(DataFrame["text"]).split()).most_common(10) #take the most common words in the dataframe  along with how many times they appear
    MostCommonStr = [] #the above call returns a list of tuples
    MostCommonValues = []
    MostCommonStr,MostCommonValues = SplitTuple(MostCommon) #split the tuple into 2 lists
    ExplodeData = (0.1, 0.15, 0.2, 0.3, 0.05, 0.25, 0.3,0.4, 0.4 ,0.0)
    Colors = ( "orange", "cyan", "brown",
          "grey", "indigo", "beige","yellow","red","pink","blue") #some nice colors again
    Preview = "Most common words in the DataFrame" #a meh title
    CreatePlot(ExplodeData,Colors,MostCommonValues,MostCommonStr,MostCommonStr,Preview)#let create plot handle the rest

In [ ]:
    #Data analysis 3
    MostCommonStr = []
    MostCommonValues = [] 
    Neg = DataFrame.groupby('sentiment').get_group("NEG") #group by sentiment
    Pos = DataFrame.groupby('sentiment').get_group("POS")
    Neu = DataFrame.groupby('sentiment').get_group("NEU")
    MostCommonNeg = Counter(" ".join(Neg["text"]).split()).most_common(3) #take the 3 most common words by each sentiment
    MostCommonPos = Counter(" ".join(Pos["text"]).split()).most_common(3) #this method returns a list of tuples
    MostCommonNeu = Counter(" ".join(Neu["text"]).split()).most_common(3) #within each there are the words and the times they appear in the dataframe
    MostCommonNegStr,MostCommonNegValues = SplitTuple(MostCommonNeg) #split the given tuples
    MostCommonPosStr,MostCommonPosValues = SplitTuple(MostCommonPos)
    MostCommonNeuStr,MostCommonNeuValues = SplitTuple(MostCommonNeu)
    for items in MostCommonNegStr: #append the most common words in tweets with negative sentiments into a list
        MostCommonStr.append(items) 
    for items in MostCommonNegValues: #append the time each word appears in the dataframe
        MostCommonValues.append(items)
    for items in MostCommonPosStr: #append the most common words in tweets with positive sentiments into a list
        MostCommonStr.append(items)
    for items in MostCommonPosValues: #append the time each word appears in the dataframe
        MostCommonValues.append(items)
    for items in MostCommonNeuStr: #append the most common words in tweets with neutral sentiments into a list
        MostCommonStr.append(items)
    for items in MostCommonNeuValues: #append the time each word appears in the dataframe
        MostCommonValues.append(items)
    SentimentArray = ["NEG is red","POS is cyan","NEU is orange"]
    ExplodeData = (0.1,0.1,0.1, 0.0,0.0,0.0, 0.2,0.2,0.2)
    Colors = ( "red","red","red","cyan","cyan","cyan","orange","orange","orange",) #some cool colors
    Preview = "Most common words in the DataFrame by sentiment" #a nice title
    CreatePlot(ExplodeData,Colors,MostCommonValues,MostCommonStr,SentimentArray,Preview) #create plot handles the rest

In [ ]:
    #Data analysis 4
    SentimentValues = []
    SentimentStr = []
    Astra = DataFrame.loc[DataFrame["text"].str.contains("astrazeneca",case=True)] #find all the tweets containing the words astrazeneca
    Pfizer = DataFrame.loc[DataFrame["text"].str.contains(pat = "moderna|pfizer|biontech",case=True)]  #find all the tweets containing the words moderna pfizer or biontech
    AstraNeg = Astra.groupby('sentiment').get_group("NEG") #group each group by sentiment
    AstraPos = Astra.groupby('sentiment').get_group("POS")
    AstraNeu = Astra.groupby('sentiment').get_group("NEU")
    PfizerNeg = Pfizer.groupby('sentiment').get_group("NEG")
    PfizerPos = Pfizer.groupby('sentiment').get_group("POS")
    PfizerNeu = Pfizer.groupby('sentiment').get_group("NEU")
    SentimentValues.append(len(AstraNeg)) #append to the values list the length of each group
    SentimentValues.append(len(PfizerNeg))
    SentimentValues.append(len(AstraNeu))
    SentimentValues.append(len(PfizerNeu))
    SentimentValues.append(len(AstraPos))
    SentimentValues.append(len(PfizerPos))
    SentimentStr.append("Astra NEG") #append the sentiment and the group name in the list
    SentimentStr.append("Pfizer NEG")
    SentimentStr.append("Astra NEU")
    SentimentStr.append("Pfizer NEU")
    SentimentStr.append("Astra POS")
    SentimentStr.append("Pfizer POS")
    Colors = ('orange','yellow','orange','yellow','orange','yellow') #nice colors
    ExplodeData = (0.1,0.1,0.1,0.2,0.2,0.2) 
    SentimentArray = ["Astra is orange","Pfizer etc. is yellow"]
    Preview = "Sentiment Comparison between Astrazeneca tweets and Pfizer etc"#a really bad title
    CreatePlot(ExplodeData,Colors,SentimentValues,SentimentStr,SentimentArray,Preview) #let create plot handle the rest

In [ ]:
    #Data analysis 5
    DataFrame['month'] = pd.DatetimeIndex(DataFrame['date']).month #Create a column in the dataframe containing the month the tweet was sent
    ExplodeData = (0.1, 0.0, 0.2, 0.3, 0.0, 0.0, 0.1,0.2, 0.2 ,0.0,0.2,0.3)
    Colors = ( "orange", "cyan", "brown",
          "grey", "indigo", "beige","purple","red","pink","blue","yellow","green") #more cool colors
    Preview = "Tweets per month" #a nice title
    Date = [dict() for i in range(12)]
    DateStr = ["Jan","Feb","March","April","May","June","July","Aug","Sept","Oct","Nov","Dec"] #all the months in a list
    DateValues = []
    for i in range(1,13):
        Date[i - 1] = DataFrame.groupby('month').get_group(i)#get for each month all the tweets sent
        DateValues.append(len(Date[i - 1])) #get the length of the column of the above dictionary
    CreatePlot(ExplodeData,Colors,DateValues,DateStr,DateStr,Preview) #create the plot
    AverageTweets = int(len(DataFrame)/12) #get the average tweets sent
    ImportantMonths = []
    for i in range(1,13):
        if DateValues[i - 1] > AverageTweets:
            ImportantMonths.append(i) #find the important months(those who are above the average tweets)
    #the rest of the data analysis 5 was not implemented

In [ ]:
#DataFrame split

In [14]:
Train, Test = sklearn.model_selection.train_test_split(DataFrame,test_size=0.2,random_state=42, shuffle=True) #split the dataframe(for some reason the DataFrame variable is unrecognized but the code runs correctly)
train_path = "Train.tsv"
test_path = "Test.tsv"
Train.to_csv(train_path,sep='\t') #save as tsv by using the tab as a separator parameter
Test.to_csv(test_path, sep='\t')

In [35]:
#Vectorization of the tweets

In [18]:
#Bag of words vectorizer
from sklearn.feature_extraction.text import CountVectorizer
Train = OpenFile("Train.tsv") #open  the tsv files
Test = OpenFile("Test.tsv")
vectorizer = CountVectorizer(max_df=1.0, min_df=1, max_features=1000,stop_words='english') #create a count vectorizer
TrainVector = vectorizer.fit_transform(Train['text']) #transform the texts in train and test dataframes
TestVector = vectorizer.transform(Test['text'])
dfTrain_bow_sklearn = pd.DataFrame(TrainVector.toarray(),columns=vectorizer.get_feature_names_out()) #create dataframes by the numpy arrays created
dfTest_bow_sklearn = pd.DataFrame(TestVector.toarray(),columns=vectorizer.get_feature_names_out())
SaveFile(dfTrain_bow_sklearn,'bagofwordsTrain.pkl') #save as pickle files in order to save time
SaveFile(dfTest_bow_sklearn,'bagofwordsTest.pkl')

In [19]:
#TF-IDF Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
Train = OpenFile("Train.tsv") #of the tsv files
Test = OpenFile("Test.tsv")
tfidf_vectorizer = TfidfVectorizer(max_df=1.0, min_df=1, max_features=1000,stop_words='english') #set parameterers for the vectorizer
tfidfTrain_vector = tfidf_vectorizer.fit_transform(Train['text']) #transform the text columns
tfidfTest_vector = tfidf_vectorizer.transform(Test['text'])
tfidfTrain_df = pd.DataFrame(tfidfTrain_vector.toarray(),columns=tfidf_vectorizer.get_feature_names_out()) #create a dataframe from that transformation
tfidfTest_df = pd.DataFrame(tfidfTest_vector.toarray(),columns=tfidf_vectorizer.get_feature_names_out())
SaveFile(tfidfTrain_df,'TF-IDFTrain.pkl') #save the dataframes to pickle files so they can easily be accessed
SaveFile(tfidfTest_df,'TF-IDFTest.pkl')